# Une représentation des inégalités scolaires










## Préliminaire - Installation des packages nécessaires 

In [24]:
!pip install pathlib2
!pip install python-Levenshtein
!pip install --upgrade xlrd
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data.git


  Cloning https://github.com/InseeFrLab/Py-Insee-Data.git to /tmp/pip-req-build-g40on3fy
  Running command git clone --filter=blob:none --quiet https://github.com/InseeFrLab/Py-Insee-Data.git /tmp/pip-req-build-g40on3fy
  Resolved https://github.com/InseeFrLab/Py-Insee-Data.git to commit 02976e01cd0e8e99630e43590910692fbde790bc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [25]:
import requests
url = 'https://github.com/InseeFrLab/Py-Insee-Data/archive/refs/heads/master.zip'
r = requests.get(url)
with open("pynsee.zip" , 'wb') as zipfile:
    zipfile.write(r.content)


In [49]:
!pip install --ignore-installed pynsee.zip
!pip install python-Levenshtein
!pip install openpyxl


Processing ./pynsee.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Shapely-1.8.5.post1-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.0 MB)
  Using cached pandas-1.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Using cached urllib3-1.26.13-py2.py3-none-any.whl (140 kB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached Unidecode-1.3.6-py3-none-any.whl (235 kB)
  Using cached pathlib-1.0.1-py3-none-any.whl (14 kB)
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached pytz-2022.6-py2.py3-none-any.whl (498 kB)
  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached certifi-2022.9.24-py3-none-any.whl (161 kB)
  Using cached idn

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pynsee.download


## I - Ouverture et manipulation des bases de données

### A - La base de données sur les résultats par données

On ouvre cette base de données à partir du fichier stocké dans le Datalab.

On s'intéresse d'abord à la feuille contenant les données pour les lycées généraux et technologiques. 
On supprime les colonnes qui ne nous seront pas utiles (sur les taux attendus et les effectifs).

In [51]:
#file_name = "ival-2021---complet-xlsx-112652.xlsx"
df = pd.read_excel ("ival-2021---complet-xlsx-112652.xlsx", header = [0,1])
#print(df.head())

#df.loc[:,("Informations établissement", "UAI")]

fichier_LGT = df.copy()
fichier_LGT = fichier_LGT.drop (["Valeur ajoutée du taux d'accès"], axis = 1)
fichier_LGT = fichier_LGT.drop(["Taux d'accès attendus", "Taux d'accès bruts", "Effectifs à la rentrée 2021", "Taux de mentions attendus", "Taux de réussite attendus"], axis = 1)

/tmp/ipykernel_130/635733873.py:8: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  fichier_LGT = fichier_LGT.drop (["Valeur ajoutée du taux d'accès"], axis = 1)
/tmp/ipykernel_130/635733873.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  fichier_LGT = fichier_LGT.drop(["Taux d'accès attendus", "Taux d'accès bruts", "Effectifs à la rentrée 2021", "Taux de mentions attendus", "Taux de réussite attendus"], axis = 1)


Les données concernant les bacs technologiques étant divisées par filière, nous regroupons toutes les filières sous une colonne technologique. 

On commence par remplacer les valeurs non existantes par des zéros. 
Puis, on calcule le nombre d'élèves présents au bac technologique. 

In [53]:
fichier_LGT = fichier_LGT.fillna(0)
#fichier.loc[:,("Nombre d'élèves présents au Bac","Somme Pro")] = fichier["Nombre d'élèves présents au Bac", "STMG"] + fichier["Nombre d'élèves présents au Bac", "STI2D"] + fichier["Nombre d'élèves présents au Bac", "STD2A"]+fichier["Nombre d'élèves présents au Bac", "STL"]+df_rose["Nombre d'élèves présents au Bac", "ST2S"]+fichier["Nombre d'élèves présents au Bac", "S2TMD"]+fichier["Nombre d'élèves présents au Bac", "STHR"]#
fichier_LGT.loc[:,("Nombre d'élèves présents au Bac","Somme Techno")] = fichier_LGT["Nombre d'élèves présents au Bac", 'TOTAL']- fichier_LGT["Nombre d'élèves présents au Bac", "GNLE"]
fichier_LGT["Nombre d'élèves présents au Bac"]


,GNLE,STMG,STI2D,STD2A,STL,ST2S,S2TMD,STHR,TOTAL,Somme Techno
0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54,0.0
1,122.0,28.0,0.0,0.0,0.0,50.0,0.0,0.0,200,78.0
2,98.0,0.0,41.0,0.0,26.0,0.0,0.0,0.0,165,67.0
3,174.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,234,60.0
4,160.0,0.0,54.0,0.0,0.0,0.0,0.0,0.0,214,54.0
...,...,...,...,...,...,...,...,...,...,...
2311,246.0,105.0,0.0,0.0,47.0,56.0,0.0,0.0,454,208.0
2312,244.0,116.0,0.0,0.0,0.0,0.0,0.0,0.0,360,116.0
2313,325.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,325,0.0
2314,221.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,251,30.0


On calcule les totaux de taux de réussite bruts pour les filières technologiques. 

In [54]:
#test = ((fichier_LGT["Taux de réussite bruts", "TOTAL"]*fichier_LGT["Nombre d'élèves présents au Bac", "TOTAL"])-(fichier_LGT["Taux de réussite bruts", "GNLE"]*fichier_LGT["Nombre d'élèves présents au Bac", "GNLE"]))/fichier_LGT["Nombre d'élèves présents au Bac", "Somme Techno"]
#fichier_LGT.loc[(fichier_LGT["Nombre d'élèves présents au Bac","Somme Techno"] > 0.5),("Taux de réussite bruts", "Taux brut techno")] = (fichier_LGT["Taux de réussite bruts", "TOTAL"]*fichier_LGT["Nombre d'élèves présents au Bac", "TOTAL"] - fichier_LGT["Taux de réussite bruts", "GNLE"]*fichier_LGT["Nombre d'élèves présents au Bac", "GNLE"])/fichier_LGT["Nombre d'élèves présents au Bac", "Somme Techno"]#
stmg = fichier_LGT["Taux de réussite bruts", "STMG"]*fichier_LGT["Nombre d'élèves présents au Bac", "STMG"]
sti2d = fichier_LGT["Taux de réussite bruts", "STI2D"]*fichier_LGT["Nombre d'élèves présents au Bac", "STI2D"]
std2a = fichier_LGT["Taux de réussite bruts", "STD2A"]*fichier_LGT["Nombre d'élèves présents au Bac", "STD2A"]
stl = fichier_LGT["Taux de réussite bruts", "STL"]*fichier_LGT["Nombre d'élèves présents au Bac", "STL"]
st2s = fichier_LGT["Taux de réussite bruts", "ST2S"]*fichier_LGT["Nombre d'élèves présents au Bac", "ST2S"]
s2tmd = fichier_LGT["Taux de réussite bruts", "S2TMD"]*fichier_LGT["Nombre d'élèves présents au Bac", "S2TMD"]
sthr = fichier_LGT["Taux de réussite bruts", "STHR"]*fichier_LGT["Nombre d'élèves présents au Bac", "STHR"]
taux_tech_brut = (stmg + sti2d + std2a + stl + st2s + s2tmd + sthr)/fichier_LGT["Nombre d'élèves présents au Bac","Somme Techno"]


fichier_LGT.loc[(fichier_LGT["Nombre d'élèves présents au Bac","Somme Techno"] > 0.5),("Taux de réussite bruts", "Taux brut techno")] = taux_tech_brut #>0.5 pour éviter erreurs d'arrondi de flottants#
fichier_LGT["Taux de réussite bruts"]
#pb avec test à cause des arrondis dans TOTAL (int pas float), résolu avec taux_techno #
#on ne peut pas le faire avec les taux de réussite attendus car on les a enlevé #
#on ne peut pas le faire avec la valeur ajoutée car mauvais type d’objet (on peut en tout cas le faire si besoin à partir du taux attendu et brut si on remet le taux attendu#


,GNLE,STMG,STI2D,STD2A,STL,ST2S,S2TMD,STHR,TOTAL,Taux brut techno
0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98,NaN
1,99.0,96.0,0.0,0.0,0.0,92.0,0.0,0.0,97,93.435897
2,100.0,0.0,100.0,0.0,96.0,0.0,0.0,0.0,99,98.447761
3,97.0,93.0,0.0,0.0,0.0,0.0,0.0,0.0,96,93.000000
4,99.0,0.0,96.0,0.0,0.0,0.0,0.0,0.0,99,96.000000
...,...,...,...,...,...,...,...,...,...,...
2311,99.0,99.0,0.0,0.0,96.0,91.0,0.0,0.0,98,96.168269
2312,100.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,100,99.000000
2313,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100,NaN
2314,99.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,99,100.000000


On calcule ces mêmes totaux pour les valeurs ajoutées. La difficulté réside dans le format des données dans les colonnes valeur ajoutée.

In [55]:
#Il faut remplacer les taux non définis par des NaN et transformer en flottant#
liste= ["GNLE", "STMG", "STI2D", "STD2A", "STL", "ST2S", "S2TMD", "STHR", "TOTAL"]
for i in liste:
    fichier_LGT.loc[fichier_LGT["Valeur ajoutée du taux de réussite", i] == "ND", ("Valeur ajoutée du taux de réussite", i)] = np.NaN
    fichier_LGT["Valeur ajoutée du taux de réussite", i] = pd.to_numeric(fichier_LGT["Valeur ajoutée du taux de réussite", i])
fichier_LGT["Valeur ajoutée du taux de réussite"].info()
fichier_LGT["Valeur ajoutée du taux de réussite"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2316 entries, 0 to 2315
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   GNLE    2301 non-null   float64
 1   STMG    2295 non-null   float64
 2   STI2D   2309 non-null   float64
 3   STD2A   2314 non-null   float64
 4   STL     2294 non-null   float64
 5   ST2S    2308 non-null   float64
 6   S2TMD   2311 non-null   float64
 7   STHR    2312 non-null   float64
 8   TOTAL   2301 non-null   float64
dtypes: float64(9)
memory usage: 163.0 KB


,GNLE,STMG,STI2D,STD2A,STL,ST2S,S2TMD,STHR,TOTAL
0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
1,2.0,5.0,0.0,0.0,0.0,-5.0,0.0,0.0,1.0
2,1.0,0.0,2.0,0.0,-1.0,0.0,0.0,0.0,1.0
3,-1.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
4,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
2311,0.0,1.0,0.0,0.0,-1.0,-5.0,0.0,0.0,0.0
2312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2314,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0


In [59]:
#calcul de la valeur ajoutée du taux de réussite pour les bacs techno, reprend tout pour éviter les problèmes d'arrondis#
stmg = fichier_LGT["Valeur ajoutée du taux de réussite", "STMG"]*fichier_LGT["Nombre d'élèves présents au Bac", "STMG"]
sti2d = fichier_LGT["Valeur ajoutée du taux de réussite", "STI2D"]*fichier_LGT["Nombre d'élèves présents au Bac", "STI2D"]
std2a = fichier_LGT["Valeur ajoutée du taux de réussite", "STD2A"]*fichier_LGT["Nombre d'élèves présents au Bac", "STD2A"]
stl = fichier_LGT["Valeur ajoutée du taux de réussite", "STL"]*fichier_LGT["Nombre d'élèves présents au Bac", "STL"]
st2s = fichier_LGT["Valeur ajoutée du taux de réussite", "ST2S"]*fichier_LGT["Nombre d'élèves présents au Bac", "ST2S"]
s2tmd = fichier_LGT["Valeur ajoutée du taux de réussite", "S2TMD"]*fichier_LGT["Nombre d'élèves présents au Bac", "S2TMD"]
sthr = fichier_LGT["Valeur ajoutée du taux de réussite", "STHR"]*fichier_LGT["Nombre d'élèves présents au Bac", "STHR"]
val_aj_tech = (stmg + sti2d + std2a + stl + st2s + s2tmd + sthr)/fichier_LGT["Nombre d'élèves présents au Bac","Somme Techno"]


fichier_LGT.loc[(fichier_LGT["Nombre d'élèves présents au Bac","Somme Techno"] > 0.5),("Valeur ajoutée du taux de réussite", "Valeur ajoutée techno")] = val_aj_tech #>0.5 pour éviter les problèmes d'arrondis de flottants#
fichier_LGT["Valeur ajoutée du taux de réussite"]

,GNLE,STMG,STI2D,STD2A,STL,ST2S,S2TMD,STHR,TOTAL,Valeur ajoutée techno
0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,NaN
1,2.0,5.0,0.0,0.0,0.0,-5.0,0.0,0.0,1.0,-1.410256
2,1.0,0.0,2.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.835821
3,-1.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-2.000000
4,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.000000
...,...,...,...,...,...,...,...,...,...,...
2311,0.0,1.0,0.0,0.0,-1.0,-5.0,0.0,0.0,0.0,-1.067308
2312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2314,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,2.000000


On passe ensuite à la feuille concernant les lycées professionnels. 

In [52]:
df2 = pd.read_excel ("ival-2021---complet-xlsx-112652.xlsx", sheet_name = 1, header = [0,1])
#print(df2.head())

fichier_LP = df2.copy()
fichier_LP = fichier_LP.drop (["Valeur ajoutée du taux d'accès", "Taux d'accès attendus", "Taux d'accès bruts", "Effectifs à la rentrée 2021", "Taux de mentions attendus", "Taux de réussite attendus"], axis = 1)
#print(fichier_LP.head())

/tmp/ipykernel_130/2210916688.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  fichier_LP = fichier_LP.drop (["Valeur ajoutée du taux d'accès", "Taux d'accès attendus", "Taux d'accès bruts", "Effectifs à la rentrée 2021", "Taux de mentions attendus", "Taux de réussite attendus"], axis = 1)


On réunit les 2 bases de données.

In [56]:
dftotale = fichier_LGT.merge(fichier_LP, how='outer', on=[('Informations établissement','UAI')])
print(display(dftotale.head(20)))

Informations établissement Informations établissement_x  \
                          UAI                     Académie   
0                    0040003G                AIX-MARSEILLE   
1                    0040027H                AIX-MARSEILLE   
2                    0040490L                AIX-MARSEILLE   
3                    0040010P                AIX-MARSEILLE   
4                    0040533H                AIX-MARSEILLE   
5                    0040023D                AIX-MARSEILLE   
6                    0133425C                AIX-MARSEILLE   
7                    0130001F                AIX-MARSEILLE   
8                    0133525L                AIX-MARSEILLE   
9                    0131319N                AIX-MARSEILLE   
10                   0130002G                AIX-MARSEILLE   
11                   0131596P                AIX-MARSEILLE   
12                   0131862D                AIX-MARSEILLE   
13                   0131391S                AIX-MARSEILLE   
14                   0131320P                AIX-MARSEILLE   
15                   0133395V                AIX-MARSEILLE   
16                   0130003H                AIX-MARSEILLE   
17                   0130010R                AIX-MARSEILLE   
18                   0130011S                AIX-MARSEILLE   
19                   0131549N                AIX-MARSEILLE   

                                                                      \
              Département                              Etablissement   
0   ALPES DE HTE PROVENCE  LYCEE ANDRE HONNORAT (GENERAL ET TECHNO.)   
1   ALPES DE HTE PROVENCE                 LYCEE ALEXANDRA DAVID NEEL   
2   ALPES DE HTE PROVENCE              LYCEE PIERRE-GILLES DE GENNES   
3   ALPES DE HTE PROVENCE                      LYCEE FELIX ESCLANGON   
4   ALPES DE HTE PROVENCE      LYCEE LES ISCLES (GENERAL ET TECHNO.)   
5   ALPES DE HTE PROVENCE      LYCEE PAUL ARENE (GENERAL ET TECHNO.)   
6        BOUCHES DU RHONE                               LYCEE CELONY   
7        BOUCHES DU RHONE      LYCEE EMILE ZOLA (GENERAL ET TECHNO.)   
8        BOUCHES DU RHONE                         LYCEE GEORGES DUBY   
9        BOUCHES DU RHONE                        LYCEE NATIVITE (LA)   
10       BOUCHES DU RHONE                         LYCEE PAUL CEZANNE   
11       BOUCHES DU RHONE                      LYCEE POILUS (BD DES)   
12       BOUCHES DU RHONE                     LYCEE SACRE COEUR (LE)   
13       BOUCHES DU RHONE         LYCEE ST ELOI (GENERAL ET TECHNO.)   
14       BOUCHES DU RHONE              LYCEE STE CATHERINE DE SIENNE   
15       BOUCHES DU RHONE       LYCEE STE MARIE (GENERAL ET TECHNO.)   
16       BOUCHES DU RHONE    LYCEE VAUVENARGUES (GENERAL ET TECHNO.)   
17       BOUCHES DU RHONE      LYCEE MONTMAJOUR (GENERAL ET TECHNO.)   
18       BOUCHES DU RHONE                              LYCEE PASQUET   
19       BOUCHES DU RHONE                LYCEE FREDERIC JOLIOT-CURIE   

                                                    \
              Ville       UAI Code commune Secteur   
0     BARCELONNETTE  0040003G        04019      PU   
1   DIGNE LES BAINS  0040027H        04070      PU   
2   DIGNE LES BAINS  0040490L        04070      PU   
3          MANOSQUE  0040010P        04112      PU   
4          MANOSQUE  0040533H        04112      PU   
5          SISTERON  0040023D        04209      PU   
6   AIX EN PROVENCE  0133425C        13001      PR   
7   AIX EN PROVENCE  0130001F        13001      PU   
8   AIX EN PROVENCE  0133525L        13001      PU   
9   AIX EN PROVENCE  0131319N        13001      PR   
10  AIX EN PROVENCE  0130002G        13001      PU   
11  AIX EN PROVENCE  0131596P        13001      PU   
12  AIX EN PROVENCE  0131862D        13001      PR   
13  AIX EN PROVENCE  0131391S        13001      PR   
14  AIX EN PROVENCE  0131320P        13001      PR   
15  AIX EN PROVENCE  0133395V        13001      PR   
16  AIX EN PROVENCE  0130003H        13001      PU   
17            ARLES  013001

None


In [57]:
df_communes = pd.read_excel("FILO2019_DEC_COM.xlsx", sheet_name = 1, header=[4,5])
df_com = df_communes.copy()
df_com = df_com.drop(['PMIMP19', 'Q119', 'Q319','Q3_Q1','D119', 'D219', 'D319', 'D419', 'D619', 'D719', 'D819', 'D919', 'RD', 'S80S2019', 'GI19', 'PACT19', 'PTSA19', 'PCHO19', 'PBEN19', 'PPEN19', 'PAUT19'], axis = 1, level = 1 )
df_com
#df_com.set_index('Code géographique)

,Code géographique,Libellé géographique,Nbre de ménages fiscaux,Nbre de personnes dans les ménages fiscaux,Nbre d'unités de consommation dans les ménages fiscaux,Médiane (€)
,CODGEO,LIBGEO,NBMEN19,NBPERS19,NBUC19,Q219
0,01001,L'Abergement-Clémenciat,322,819,542.7,24210
1,01002,L'Abergement-de-Varey,114,267,179.5,23380
2,01004,Ambérieu-en-Bugey,6713,15041,10298.9,19690
3,01005,Ambérieux-en-Dombes,720,1823,1204.3,24100
4,01007,Ambronay,1163,2857,1892.3,23340
...,...,...,...,...,...,...
31359,97420,Sainte-Suzanne,8346,23842,15051.4,14100
31360,97421,Salazie,2469,6936,4407.0,7740
31361,97422,Le Tampon,30083,78013,50752.2,12370


In [58]:
df_com.columns = df_com.columns.map('_'.join).str.strip('_')
dftotale.columns = dftotale.columns.map('_'.join).str.strip('_')


## II - Les premières statistiques descriptives